In [66]:
import torch
import numpy
import math
import numpy as np
from scipy.stats import multivariate_normal as normal
import matplotlib.pyplot as plt

In [67]:
class one_time_net(torch.nn.Module):
    def __init__(self,d,acti):
        super(one_time_net,self).__init__()
        self.n_neuron=[d,d+10,d+10,1]
        self.norm=torch.nn.BatchNorm1d(self.n_neuron[0])
        self.acti=acti
        self.layers=torch.nn.ModuleList([self._one_layer(self.n_neuron[0],self.n_neuron[1],torch.nn.ReLU()),self._one_layer(self.n_neuron[1],self.n_neuron[2],torch.nn.ReLU()),
                                        self._one_layer(self.n_neuron[2],self.n_neuron[3],acti)])
    def _one_layer(self,input_dim,output_dim,activation_fn=torch.nn.ReLU()):
        if activation_fn != None:
            return torch.nn.Sequential(torch.nn.Linear(input_dim,output_dim),torch.nn.BatchNorm1d(output_dim),activation_fn)
        else:
            return torch.nn.Sequential(torch.nn.Linear(input_dim,output_dim),torch.nn.BatchNorm1d(output_dim))
    def forward(self,x):
        norm=self.norm(x)
        l1=self.layers[0](x)
        l2=self.layers[1](l1)
        out=self.layers[2](l2)
        return out       

In [68]:
#model parameters
r=0.03 #interest rate
mu=0.07 #average return
sigma=0.3 #volatility
#lam=1
#jump=-0.1
T=1 #terminal time
c1=1.2 #terminal utility parameter1
c2=0 #terminal utility parameter2
xref=0#terminal utility parameter3
gamma=0.5 #risk aversion
d=0 #lower bound of strategy
u=1 #upper bound of strategy
zmin=1e-4
zmax=2
N=10
deltat=T/N
sqrt=np.sqrt(deltat)
eta=mu-r
pi0=eta/sigma**2/gamma
pistar=d*(pi0<d)+pi0*(pi0>=d)*(pi0<=u)+u*(pi0>u)
AA=eta*pistar-gamma/2*pistar**2*sigma**2

In [69]:
class wealthmodel(object):
    def __init__(self,mu,r,sigma,deltat):
        self.mu=mu
        self.r=r
        self.sigma=sigma
        self.deltat=deltat
        self.sqrt=np.sqrt(deltat)
        #self.noise=torch.distributions.normal.Normal(torch.tensor([0.0]),torch.tensor([1.0]))
    def sample(self,w,pi,batch_size):
        noise=torch.distributions.multivariate_normal.MultivariateNormal(torch.zeros(batch_size),torch.eye(batch_size))
        next_w=w*((self.mu-self.r)*self.deltat*pi+self.sigma*pi*self.sqrt*noise.sample().reshape([batch_size,1]))+w
        return next_w
    def utility(self,w):
        return w**(1-gamma)/(1-gamma)*(w<xref).float()+(c1*w**(1-gamma)/(1-gamma)+c2)*(w>=xref).float()

In [99]:
batch_size=512
lr=5e-4
model=wealthmodel(mu,r,sigma,deltat)
N_iter=16000
values=[]
controls=[]
for i in range(N):
    values.append(one_time_net(1,None))
    controls.append(one_time_net(1,torch.nn.Sigmoid()))
voptims=[]
coptims=[]
for i in range(N):
    voptims.append(torch.optim.Adam(values[i].parameters(),lr=lr))
    coptims.append(torch.optim.Adam(controls[i].parameters(),lr=lr))
criterion=torch.nn.MSELoss()
noise=torch.distributions.multivariate_normal.MultivariateNormal(torch.zeros(batch_size),torch.eye(batch_size))

In [100]:
for i in list(controls):
    for name,para in i.named_parameters():
        if name=='layers.2.1.bias':
            torch.nn.init.constant_(para,np.log(pistar/(1-pistar)))
        elif 'layers' in name:
            #torch.nn.init.constant_(para, 0)
            torch.nn.init.normal_(para, std=.01)

In [ ]:
for i in range(N_iter):
    w=(zmax-zmin)*torch.rand([batch_size,1])+zmin
    pi=(u-d)*controls[-1](w)+d
    brown=noise.sample().reshape([batch_size,1])
    next_w=w*torch.exp((mu-r)*deltat*pi-0.5*sigma**2*pi**2*deltat+pi*sigma*sqrt*brown)
    target=model.utility(next_w)
    policy_loss=-1*target.mean()
    coptims[-1].zero_grad()
    policy_loss.backward()
    coptims[-1].step()
    v=values[-1](w)
    loss=criterion(v,target.detach())
    voptims[-1].zero_grad()
    loss.backward()
    voptims[-1].step()


In [56]:
for t in reversed(range(N-1)):
    for i in range(N_iter):
        w=(zmax-zmin)*torch.rand([batch_size,1])+zmin
        pi=(u-d)*controls[t](w)+d
        brown=noise.sample().reshape([batch_size,1])
        next_w=w*((mu-r)*deltat*pi+sigma*pi*sqrt*brown)+w
        target=values[t+1](next_w)
        policy_loss=-1*target.mean()
        coptims[t].zero_grad()
        policy_loss.backward()
        coptims[t].step()
        v=values[t](w)
        loss=criterion(v,target.detach())
        voptims[t].zero_grad()
        loss.backward()
        voptims[t].step()    

In [ ]:
t=N-1
zgrid=np.linspace(zmin,zmax,1001)
z=torch.tensor(zgrid).reshape([-1,1]).float()
pi=(u-d)*controls[t](z)+d
v=values[t](z)

In [ ]:
plt.plot(zgrid,pi.detach().numpy().reshape([-1,]))
plt.plot(zgrid,pi0*np.ones(shape=zgrid.shape))

In [ ]:
plt.plot(zgrid,v.detach().numpy().reshape([-1,]))
plt.plot(zgrid,model.utility(z).detach().numpy().reshape([-1,])*np.exp(-1*AA*(T-t*deltat)))

In [ ]:
for name,p in controls[-1].named_parameters():
    print(name,p)
    if name=='layers.2.1.bias':
        print(p)

In [97]:
pistar

0.8888888888888891